#### Loading in the data

In [10]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler

In [11]:
ticker = 'GC=F'  # Gold Futures ticker on Yahoo Finance
gold = yf.download(ticker, start='2016-01-01', end='2025-01-01', interval='1d')
gold.head()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,GC=F,GC=F,GC=F,GC=F,GC=F
Date,,,,,
2016-01-04,1075.099976,1082.500000,1063.199951,1063.400024,143
2016-01-05,1078.400024,1081.500000,1075.300049,1075.599976,82
2016-01-06,1091.900024,1093.699951,1081.599976,1081.599976,52
2016-01-07,1107.699951,1109.400024,1091.599976,1091.599976,122
2016-01-08,1097.800049,1111.099976,1093.000000,1111.099976,98


In [12]:
gold.to_csv('gold_prices_2016_2025.csv')

#### Checking the data


Removing the "ticker" row

In [16]:
gold.columns = gold.columns.droplevel(1)
gold.head()

Price,Close,High,Low,Open,Volume
Date,,,,,
2016-01-04,1075.099976,1082.500000,1063.199951,1063.400024,143
2016-01-05,1078.400024,1081.500000,1075.300049,1075.599976,82
2016-01-06,1091.900024,1093.699951,1081.599976,1081.599976,52
2016-01-07,1107.699951,1109.400024,1091.599976,1091.599976,122
2016-01-08,1097.800049,1111.099976,1093.000000,1111.099976,98


In [17]:
gold.describe()

Price,Close,High,Low,Open,Volume
count,2261.000000,2261.000000,2261.000000,2261.000000,2261.000000
mean,1652.980317,1660.537551,1645.281999,1652.768554,5274.795223
std,383.260426,385.458997,380.943530,383.159313,29509.349030
min,1073.900024,1081.500000,1063.199951,1063.400024,0.000000
25%,1288.400024,1292.000000,1283.599976,1287.300049,47.000000
50%,1705.000000,1715.300049,1694.699951,1704.699951,181.000000
75%,1898.599976,1907.699951,1886.099976,1897.300049,560.000000
max,2788.500000,2789.000000,2774.600098,2787.399902,386334.000000


In [18]:
gold.dtypes

Price
Close     float64
High      float64
Low       float64
Open      float64
Volume      int64
dtype: object

In [19]:
gold['Date'] = pd.to_datetime(gold['Date'])
gold.set_index('Date', inplace=True)

KeyError: 'Date'

Checking for missing values